<a href="https://colab.research.google.com/github/AmedeeRoy/BirdDL/blob/main/projection/SS_unet_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive/")

%cd drive/My\ Drive/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


# LOAD MODELS

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import sqlite3

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils.trip import *
from utils.unet import *

# Load data

In [9]:
data_all = pd.read_table('./data/gps+5-30s.csv', sep = ',')
check = pd.read_table('./data/check_gps+5-30s.csv', sep = ',')

# SS 15s
## data

In [10]:
select = check.trip[(check.species == 'SS') & (check.res == 15)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
248985,F0915_11_SS_T1,2015-09-30 04:38:50,-32.391042,-3.818819,0.0,6.490572,-7.099348,0,-2.355758,-1.404850,0.317965,0.992333,-0.123590
248986,F0915_11_SS_T1,2015-09-30 04:39:05,-32.390598,-3.818280,0.0,5.175050,-43.053026,0,-2.354386,-1.399249,0.253519,0.730722,-0.682675
248987,F0915_11_SS_T1,2015-09-30 04:39:20,-32.389938,-3.816505,0.0,14.049271,-19.077874,0,-2.352347,-1.380792,0.688255,0.945075,-0.326853
248988,F0915_11_SS_T1,2015-09-30 04:39:35,-32.389033,-3.815971,0.0,7.784404,39.019904,0,-2.349551,-1.375235,0.381348,0.776927,0.629590
248989,F0915_11_SS_T1,2015-09-30 04:39:50,-32.388054,-3.815510,0.0,8.016483,5.385739,0,-2.346526,-1.370445,0.392717,0.995585,0.093861
...,...,...,...,...,...,...,...,...,...,...,...,...,...
420369,F0917_8_SS_T1,2017-09-17 14:48:01,-32.397746,-3.820496,0.0,6.730195,6.892192,0,-1.489018,-2.050227,0.375630,0.992774,0.120002
420370,F0917_8_SS_T1,2017-09-17 14:48:16,-32.397666,-3.821204,0.0,5.285263,12.401750,0,-1.487906,-2.054250,0.294020,0.976666,0.214765
420371,F0917_8_SS_T1,2017-09-17 14:48:31,-32.397051,-3.821824,0.0,6.475610,-38.270823,0,-1.479330,-2.057777,0.361251,0.785092,-0.619379
420372,F0917_8_SS_T1,2017-09-17 14:48:46,-32.396036,-3.821856,0.0,7.521297,-43.507883,0,-1.465173,-2.057959,0.420311,0.725280,-0.688454


In [11]:
model = torch.load('SV_unet_matrixlonlatcov_15s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [12]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

F0915_11_SS_T1
F0915_11_SS_T2
F0915_12_SS_T1
F0915_12_SS_T2
F0915_13_SS_T1
F0915_14_SS_T1
F0915_15_SS_T1
F0915_15_SS_T2
F0915_2_SS_T1
F0915_2_SS_T2
F0915_22_SS_T1
F0915_23_SS_T1
F0915_23_SS_T2
F0915_24_SS_T1
F0915_24_SS_T2
F0915_3_SS_T1
F0915_3_SS_T2
F0915_3_SS_T3
F0915_30_SS_T1
F0915_31_SS_T1
F0916_1_SS_T1
F0916_1_SS_T2
F0916_10_SS_T1
F0916_10_SS_T2
F0916_12_SS_T1
F0916_13_SS_T1
F0916_14_SS_T1
F0916_15_SS_T1
F0916_17_SS_T1
F0916_17_SS_T2
F0916_2_SS_T1
F0916_29_SS_T1
F0916_3_SS_T1
F0916_4_SS_T1
F0916_4_SS_T2
F0916_5_SS_T1
F0916_5_SS_T2
F0916_6_SS_T1
F0916_7_SS_T1
F0916_9_SS_T1
F0917_1_SS_T1
F0917_10_SS_T1
F0917_10_SS_T2
F0917_2_SS_T1
F0917_2_SS_T2
F0917_4_SS_T1
F0917_5_SS_T1
F0917_6_SS_T1
F0917_7_SS_T1
F0917_7_SS_T2
F0917_8_SS_T1


In [13]:
data.to_csv('SS_15s.csv', index = False)

# SS 30s
## data

In [14]:
select = check.trip[(check.species == 'SS') & (check.res == 30)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
328964,F0916_16_SS_T1,2016-09-23 05:48:41,-32.386742,-3.815210,0.000000,9.238286,41.319280,0,-2.337623,2.850745,0.540166,0.751042,6.602544e-01
328965,F0916_16_SS_T1,2016-09-23 05:49:11,-32.384351,-3.814188,0.000000,9.629107,-3.175954,0,-2.326319,2.869042,0.563017,0.998464,-5.540248e-02
328966,F0916_16_SS_T1,2016-09-23 05:49:41,-32.381783,-3.813694,0.000000,9.680551,12.243868,0,-2.314182,2.877905,0.566025,0.977254,2.120731e-01
328967,F0916_16_SS_T1,2016-09-23 05:50:11,-32.381502,-3.812650,0.000000,4.010061,-64.004165,0,-2.312851,2.896602,0.234470,0.438306,-8.988259e-01
328968,F0916_16_SS_T1,2016-09-23 05:50:41,-32.380944,-3.811821,0.000000,3.703339,18.807409,0,-2.310216,2.911449,0.216536,0.946608,3.223881e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440461,F0918_3_SS_T2,2018-09-21 13:59:02,-31.752864,-2.362515,0.020833,0.407829,-2.848756,0,0.214461,-0.242499,0.014077,0.998764,-4.969968e-02
440462,F0918_3_SS_T2,2018-09-21 13:59:32,-31.752867,-2.362572,0.762500,0.212929,-64.701139,0,0.214443,-0.242634,0.007350,0.427340,-9.040910e-01
440463,F0918_3_SS_T2,2018-09-21 14:00:02,-31.752528,-2.362747,1.000000,1.415229,-66.061535,0,0.216234,-0.243045,0.048850,0.405755,-9.139818e-01
440464,F0918_3_SS_T2,2018-09-21 14:00:32,-31.751856,-2.363011,1.000000,2.675992,-5.795858,0,0.219777,-0.243666,0.092368,0.994888,-1.009844e-01


In [15]:
model = torch.load('SV_unet_matrixlonlatcov_30s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [16]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

F0916_16_SS_T1
F0916_16_SS_T2
F0916_27_SS_T1
F0916_28_SS_T1
F0918_1_SS_T1
F0918_14_SS_T1
F0918_14_SS_T2
F0918_2_SS_T1
F0918_3_SS_T1
F0918_3_SS_T2


In [17]:
data.to_csv('SS_30s.csv', index = False)